In [1]:
import pandas as pd
import prepare 

/usr/local/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Question 1.
Does legality effect price?

- bring in the dataset

In [2]:
df = pd.read_json('default-cards-20220727090443.json')

In [3]:
df = prepare.prepare_dataframe(df)

In [4]:
train, validate, test = prepare.split_data(df)

In [8]:
train.head()

,id,name,lang,released_at,layout,highres_image,image_status,mana_cost,cmc,type_line,...,hand_modifier,content_warning,flavor_name,usd,usd_foil,eur,eur_foil,is_foil,is_etched,is_glossy
12661,2d911053-a026-4b20-ba2d-dbcc367c1413,Ajani Goldmane,en,2010-07-16,normal,True,highres_scan,{2}{W}{W},4.0,Legendary Planeswalker — Ajani,...,0.0,False,no_flavor,8.02,15.11,8.02,15.11,True,False,False
61300,dcb2e0d6-3181-4d3d-a3b4-3896288b2e0e,Champion of Lambholt,en,2020-08-07,normal,True,highres_scan,{1}{G}{G},3.0,Creature — Human Warrior,...,0.0,False,no_flavor,0.36,3.23,0.36,3.23,True,False,False
48604,af3aa149-00fa-4932-b004-f64c8b5d3ca7,Gargoyle,en,2009-07-17,token,True,highres_scan,,0.0,Token Artifact Creature — Gargoyle,...,0.0,False,no_flavor,0.25,0.00,0.25,0.00,False,False,False
55734,c8d75604-4941-4faf-8f1a-7fc529e64962,Goblin Settler,en,2022-04-12,normal,True,highres_scan,{3}{R},4.0,Creature — Goblin,...,0.0,False,no_flavor,9.76,11.87,9.76,11.87,True,False,False
70638,feef03a1-f19a-413b-ad86-0b357cf9c09c,Weight Advantage,en,2016-08-26,normal,True,highres_scan,,0.0,Conspiracy,...,0.0,False,no_flavor,0.11,0.42,0.11,0.42,True,False,False


Now that the data is in the form that I need it in I will make the legalities change

To do so I will need to the columns I have made for the 

In [9]:
train.columns.unique()

Index(['id', 'name', 'lang', 'released_at', 'layout', 'highres_image',
       'image_status', 'mana_cost', 'cmc', 'type_line', 'oracle_text', 'power',
       'toughness', 'colors', 'color_identity', 'keywords', 'legalities',
       'games', 'reserved', 'foil', 'nonfoil', 'finishes', 'oversized',
       'promo', 'reprint', 'variation', 'set_id', 'set', 'set_name',
       'set_type', 'collector_number', 'digital', 'rarity', 'flavor_text',
       'artist', 'border_color', 'frame', 'full_art', 'textless', 'booster',
       'story_spotlight', 'edhrec_rank', 'penny_rank', 'prices', 'promo_types',
       'security_stamp', 'produced_mana', 'watermark', 'frame_effects',
       'loyalty', 'printed_name', 'color_indicator', 'life_modifier',
       'hand_modifier', 'content_warning', 'flavor_name', 'usd', 'usd_foil',
       'eur', 'eur_foil', 'is_foil', 'is_etched', 'is_glossy'],
      dtype='object')

In [10]:
def make_card_type_column(df):
    
    card_types = ['Creature', 'Land', 'Enchantment', 'Artifact', 'Sorcery', 'Instant', 'Legendary', 'Token']

    for card_type in card_types:
        df[f'is_{card_type.lower()}'] = df.type_line.str.contains(card_type).fillna(False)

    layouts = ['normal','token','art_series','transform','modal_dfc','split','planar','adventure','vanguard','saga',
    'emblem','scheme','double_faced_token','leveler','flip','class','host','meld','augment','reversible_card']
    
    for layout in layouts:
        df[f'is_{layout.lower()}'] = df.type_line.str.contains(layout).fillna(False)
    df[f'is_split_card'] = df.type_line.str.contains('Card // Card').fillna(False)
    return df

In [11]:
train = make_card_type_column(train)
train.head()

/var/folders/cs/9mn89l_n6p3c9sq01q5f54kh0000gn/T/ipykernel_6608/3945789389.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'is_{card_type.lower()}'] = df.type_line.str.contains(card_type).fillna(False)
/var/folders/cs/9mn89l_n6p3c9sq01q5f54kh0000gn/T/ipykernel_6608/3945789389.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'is_{layout.lower()}'] = df.type_line.str.contains(layout).fillna(False)
/var/folders/cs/9mn89l_n6p3c9sq01q5f54kh0000gn/T/ipykernel_6608/3945789389.py:11: SettingWithCopy

,id,name,lang,released_at,layout,highres_image,image_status,mana_cost,cmc,type_line,...,is_scheme,is_double_faced_token,is_leveler,is_flip,is_class,is_host,is_meld,is_augment,is_reversible_card,is_split_card
12661,2d911053-a026-4b20-ba2d-dbcc367c1413,Ajani Goldmane,en,2010-07-16,normal,True,highres_scan,{2}{W}{W},4.0,Legendary Planeswalker — Ajani,...,False,False,False,False,False,False,False,False,False,False
61300,dcb2e0d6-3181-4d3d-a3b4-3896288b2e0e,Champion of Lambholt,en,2020-08-07,normal,True,highres_scan,{1}{G}{G},3.0,Creature — Human Warrior,...,False,False,False,False,False,False,False,False,False,False
48604,af3aa149-00fa-4932-b004-f64c8b5d3ca7,Gargoyle,en,2009-07-17,token,True,highres_scan,,0.0,Token Artifact Creature — Gargoyle,...,False,False,False,False,False,False,False,False,False,False
55734,c8d75604-4941-4faf-8f1a-7fc529e64962,Goblin Settler,en,2022-04-12,normal,True,highres_scan,{3}{R},4.0,Creature — Goblin,...,False,False,False,False,False,False,False,False,False,False
70638,feef03a1-f19a-413b-ad86-0b357cf9c09c,Weight Advantage,en,2016-08-26,normal,True,highres_scan,,0.0,Conspiracy,...,False,False,False,False,False,False,False,False,False,False


the above warnings are pretty new so I'll go in and check

In [ ]:
train.head(5)